intro 

F1

plots -> 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns',50)
pd.options.display.max_rows

60

In [ ]:
initFile = pd.read_csv('listings.csv',index_col= 'id',low_memory=False)
initFile.head(1)

In [ ]:
lst = ['listing_url','scrape_id','last_scraped','summary',\
       'space','description','experiences_offered','neighborhood_overview','interaction','access','house_rules',\
       'notes','thumbnail_url','medium_url','picture_url','host_url',\
      'host_about','host_response_rate','host_acceptance_rate','host_thumbnail_url','host_picture_url',\
       'host_listings_count','host_has_profile_pic','host_verifications',\
       'neighbourhood','neighbourhood_group_cleansed','market','country_code',\
      'country','is_location_exact','square_feet','weekly_price','monthly_price','minimum_minimum_nights',\
      'minimum_minimum_nights','maximum_minimum_nights','minimum_maximum_nights','maximum_maximum_nights',\
      'minimum_nights_avg_ntm','maximum_nights_avg_ntm','calendar_updated','has_availability','calendar_last_scraped',\
      'availability_30','availability_60','availability_90','availability_365','number_of_reviews_ltm',\
       'requires_license','license','jurisdiction_names','is_business_travel_ready','require_guest_profile_picture',\
      'require_guest_phone_verification','calculated_host_listings_count','calculated_host_listings_count_entire_homes',\
      'calculated_host_listings_count_private_rooms','calculated_host_listings_count_shared_rooms','xl_picture_url',\
      ]
initFile.drop(columns=lst,inplace=True)

In [ ]:
initFile.head()

In [ ]:
initFile.reset_index(inplace=True)

In [ ]:
initFile['colid'] = initFile.index 

In [ ]:
initFile.drop('id',axis=1,inplace=True)

In [ ]:
initFile.rename(columns={'colid':'id'},inplace=True)
initFile.id +=1

In [ ]:
initFile.set_index('id',inplace=True)
initFile.head(1)

## Cleaning the dataset to make sure we have only CA related values

In [ ]:
df = initFile.copy()

In [ ]:
df.state.loc[df.state=='California'] = 'CA'
df.state.unique()
df.state = df['state'].str.upper()

In [ ]:
df.state.value_counts()

## We need to remove the non-CA values from the dataset

In [ ]:
df.drop(df[~(df.state.values=='CA')].index,axis=0,inplace=True)

In [ ]:
df.columns

In [ ]:
df.city = df.city.str.lower()
df.city.unique()
df.city.dropna(inplace=True)

In [ ]:
df['city'] = df['city'].str.replace(r'[^\x00-\x7F]+', '')
df.city.dropna(inplace=True)
df.city.value_counts()

In [ ]:
df.groupby('city')['neighbourhood_cleansed'].value_counts()

In [ ]:
df.smart_location.shape

In [ ]:
df.smart_location = df.smart_location.str.lower()

In [ ]:
df.smart_location.unique()

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(df[(df['smart_location'] == '蒙特利帕克, ca') | \
                       (df['smart_location'] == '因達斯特里, ca') | \
                       (df['smart_location'] == '艾尔蒙地, ca') | \
                       (df['smart_location'] == '沃尔纳特, ca') | \
                       (df['smart_location'] == '哈仙达岗, ca') | \
                       (df['smart_location'] == '罗兰高地, ca') | \
                       (df['smart_location'] == '罗兰岗, ca') | \
                       (df['smart_location'] == '天普市, ca') | \
                       (df['smart_location'] == '洛杉矶, ca')  \
                      ].index, inplace = True)

In [ ]:
df.smart_location = df.smart_location.str.replace(', ca',' ', regex=True)
df.smart_location = df.smart_location.str.strip()
df.neighbourhood_cleansed = df.neighbourhood_cleansed.str.strip()

In [ ]:
df['location'] = ''

In [ ]:
df.smart_location = df.smart_location.str.lower()
df.neighbourhood_cleansed = df.neighbourhood_cleansed.str.lower() 
df.location = df.location.str.lower()
df.head()

In [ ]:
 df.location =  df.neighbourhood_cleansed[df['neighbourhood_cleansed']== df['smart_location']]
 df.head()

In [ ]:
df.location.isna().value_counts()

In [ ]:
df.location.fillna(df.neighbourhood_cleansed+','+df.smart_location,inplace=True)
df.head()

In [ ]:
df.groupby('location')['zipcode'].value_counts().sort_values(ascending=False)

In [ ]:
df.columns

## Make a dataset copy for Interesting Fact 1 which is focused on finding out the most affordable places in CA 


#### More adept for indetails about the location : 'host_name','host_location','host_response_time','host_is_superhost', 'property_type','room_type','accommodates','bathrooms','bedrooms','beds','bed_type'

In [ ]:
df1 = df[['price','security_deposit','cleaning_fee','guests_included','extra_people','minimum_nights',\
          'maximum_nights','amenities','location',\
       'state','zipcode','latitude','longitude']].copy()

In [ ]:
df1.info()

In [ ]:
df1.isna().sum()

In [ ]:
df1.amenities.value_counts().nlargest(10)

In [ ]:
df1.groupby('amenities')['price'].value_counts().nlargest(10)

# 

In [ ]:
df1.head()

In [ ]:
df1.amenities = df1.amenities.str.strip('{}')
df1.amenities = df1.amenities.str.replace('"','')
df1.amenities = df1.amenities.str.lower()

In [ ]:
#from collections import Counter
#Counter(" ".join(df1["amenities"]).split(',')).most_common()
pd.Series(' '.join(df1['amenities']).lower().split(',')).value_counts().nlargest(12)

In [ ]:
df1.amenities[df1.amenities.str.contains('washer')].values[1]

In [ ]:
df1['wifi'] = ''
df1['ac'] = ''
df1['pet'] = ''
df1['park'] = ''
df1['htub'] = ''
df1['dishwasher'] = ''
df1['bbq'] = ''
df1['pool'] = ''
df1['hr'] = ''
df1['coffee'] = ''
df1['heating'] = ''
df1['iron'] = ''
df1['washer'] = ''
df1['dryer'] = ''
df1['parking'] = ''

In [ ]:
df1.wifi = df1.amenities.str.contains("wifi").apply(lambda x: 'wifi' if x==True else '')
df1.ac = df1.amenities.str.contains("air conditioning").apply(lambda x: 'air conditioning' if x==True else '')
df1.pet = df1.amenities.str.contains("pet").apply(lambda x: 'pet' if x==True else '')
df1.park = df1.amenities.str.contains("park").apply(lambda x: 'park' if x==True else '')
df1.htub = df1.amenities.str.contains("hot tub").apply(lambda x: 'hot tub' if x==True else '')
df1.dishwasher = df1.amenities.str.contains("dishwasher").apply(lambda x: 'dishwasher' if x==True else '')
df1.bbq = df1.amenities.str.contains("bbq").apply(lambda x: 'BBQ' if x==True else '')
df1.pool = df1.amenities.str.contains("pool").apply(lambda x: 'pool' if x==True else '')
df1.hr = df1.amenities.str.contains("24-hour check-in").apply(lambda x: '24-hour check-in' if x==True else '')
df1.coffee = df1.amenities.str.contains("coffee").apply(lambda x: 'coffee' if x==True else '')
df1.heating = df1.amenities.str.contains("heating").apply(lambda x: 'heating' if x==True else '')
df1.iron = df1.amenities.str.contains("iron").apply(lambda x: 'iron' if x==True else '')

In [ ]:
df1.washer = df1.amenities.str.contains("washer").apply(lambda x: 'washer' if x==True else '')
df1.dryer = df1.amenities.str.contains("dryer").apply(lambda x: 'dryer' if x==True else '')
df1.parking = df1.amenities.str.contains("parking").apply(lambda x: 'parking' if x==True else '')

In [ ]:
df1.drop('amenities',axis=1,inplace=True)

In [ ]:
df1['amenities'] = df1.wifi+' '+df1.ac+' '+df1.pet+' '+df1.park+' '+\
df1.htub+' '+df1.dishwasher+' '+df1.bbq+' '+df1.pool+' '+df1.hr+' '+df1.coffee+' '+\
df1.heating+' '+df1.iron+' '+df1.washer+' '+df1.dryer+' '+df1.parking


In [ ]:
df1.wifi = df1.wifi.apply(lambda x: 1 if x=='wifi' else 0)
df1.ac = df1.ac.apply(lambda x: 1 if x=='air conditioning' else 0)
df1.pet = df1.pet.apply(lambda x: 1 if x=='pet' else 0)
df1.park = df1.park.apply(lambda x:1 if x=='park' else 0)
df1.htub = df1.htub.apply(lambda x:1 if x=='hot tub' else 0)
df1.dishwasher = df1.dishwasher.apply(lambda x:1 if x=='dishwasher' else 0)
df1.bbq = df1.bbq.apply(lambda x:1 if x=='BBQ' else 0)
df1.pool = df1.pool.apply(lambda x:1 if x=='pool' else 0)
df1.hr = df1.hr.apply(lambda x:1 if x=='24-hour check-in' else 0)
df1.coffee = df1.coffee.apply(lambda x:1 if x=='coffee' else 0)
df1.heating = df1.heating.apply(lambda x:1 if x=='heating' else 0)
df1.iron = df1.iron.apply(lambda x:1 if x=='iron' else 0)
df1.washer = df1.washer.apply(lambda x:1 if x=='washer' else 0)
df1.dryer = df1.dryer.apply(lambda x:1 if x=='dryer' else 0)
df1.parking = df1.parking.apply(lambda x:1 if x=='parking' else 0)


In [ ]:
df1.head()

In [ ]:
df1.isna().sum()

In [ ]:
df1.security_deposit = df1.security_deposit.str.strip('$')
df1.security_deposit = df1.security_deposit.str.replace(',','')

In [ ]:
df1.security_deposit = df1.security_deposit.astype('float64')

In [ ]:
df1.security_deposit.describe()

In [ ]:
df1.security_deposit.fillna(round(df1.security_deposit.mean(),2),inplace=True)

In [ ]:
df1.cleaning_fee.describe()

In [ ]:
df1.cleaning_fee.isna().sum()

In [ ]:
df1.cleaning_fee = df1.cleaning_fee.str.strip('$')
df1.cleaning_fee = df1.cleaning_fee.str.replace(',','')

In [ ]:
df1.cleaning_fee = df1.cleaning_fee.astype('float64')

In [ ]:
df1.cleaning_fee.describe()

In [ ]:
df1.cleaning_fee.fillna(round(df1.cleaning_fee.mean(),2),inplace=True)

In [ ]:
df1.cleaning_fee.describe()

In [ ]:
df1.isna().sum()

In [ ]:
df1.location[df1.zipcode.isna()==True]

In [ ]:
for x in df1.location[df1.zipcode.isna()==True].values:
    if x=='malibu':
        idxZip = df1.index(x)
        = df1[df1.location=='malibu'].zipcode.values[0]
        


In [ ]:
df1.price.hist()

In [ ]:
df1.groupby('price')['location'].value_counts().nsmallest(20)

In [ ]:
df1.price.describe()

In [ ]:
df1.price = df1.price.str.strip('$')
df1.price = df1.price.str.replace(',','')

In [ ]:
df1.price = df1.price.astype('float64')

In [ ]:
df1.price.describe()

In [ ]:
df1.drop(df1[df1.price==0].index,axis=0,inplace=True)

In [ ]:
df1.price.count()

In [ ]:
df1.to_csv('AffordabilityDataset.csv',index=False)

In [ ]:
df1[df1.price==1000]

Plotly graph:

import chart_studio.plotly as py
import plotly.figure_factory as ff
#from plotly.figure_factory._county_choropleth import create_choropleth


#df_sample = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/minoritymajority.csv')
df_plot = df1[df1['state'] == 'CA']

values = df_plot['price'].tolist()
fips = '06'

endpts = list(np.mgrid[min(values):max(values):4j])
colorscale = [
    'rgb(193, 193, 193)',
    'rgb(239,239,239)',
    'rgb(195, 196, 222)',
    'rgb(144,148,194)',
    'rgb(101,104,168)',
    'rgb(65, 53, 132)'
]

fig = ff.create_choropleth(
    fips=fips, values=values, scope=['California'], show_state_data=True,
    colorscale=colorscale, binning_endpoints=endpts, round_legend_values=True,
    plot_bgcolor='rgb(229,229,229)',
    paper_bgcolor='rgb(229,229,229)',
    legend_title='AirBnB Locations according to their per day prices',
    county_outline={'color': 'rgb(255,255,255)', 'width': 0.5},
    exponent_format=True,
)
py.iplot(fig, filename='choropleth_california')

Geopandas graph tool: 

import geopandas as gpd
from shapely.geometry import Point, Polygon
usa = gpd.read_file('./state_21_basic/states.shp')
usa.head()


usa[usa.STATE_ABBR == 'CA'].plot()

In [ ]:
df1.price.describe()

In [ ]:
pd.qcut(df1.price,10)

In [ ]:
df1.groupby('price')['location'].value_counts().nlargest(10)

In [ ]:
#df1.price.loc[(df1['price']<= 9.999) & (df1['price'] <= 57.0)] = 1.0 #Low
df1.price.loc[(df1['price'] > 57.0) & (df1['price'] <= 84.0)] = 1.1
df1.price.loc[(df1['price'] > 84.0) & (df1['price'] <= 110.0)] = 1.2
df1.price.loc[(df1['price'] > 110.0) & (df1['price'] <= 150.0)] = 2.0 #Mid
df1.price.loc[ df1['price'] > 150.0 & (df1['price'] <= 249.0)] = 2.1
df1.price.loc[ df1['price'] > 249.0 & (df1['price'] <= 350.0)] = 2.2
df1.price.loc[ df1['price'] > 350.0 & (df1['price'] <= 500.0)] = 3.0 #High
df1.price.loc[ df1['price'] > 500.0 & (df1['price'] <= 900.0)] = 3.1
df1.price.loc[ df1['price'] > 900.0 & (df1['price'] <= 1500.0)] = 3.2
df1.price.loc[ df1['price'] > 1500.0 & (df1['price'] <= 25000.0)] = 3.3
    


In [ ]:
import folium
from folium.plugins import MarkerCluster

In [ ]:
# Setting a base map
lat = 34.1000
long = -118.4662

base = folium.Map(location=[lat,long], zoom_start=10)
base

In [ ]:
neighbourhoods = folium.map.FeatureGroup()

In [ ]:
for inc_lat,inc_long in zip(df1.longitude,df1.latitude):
    neighbourhoods.add_child(
    folium.CircleMarker(
    [inc_long,inc_lat],
    radius = 5,
    color='yellow',
    fill=True,
    fill_color='blue',
    fill_opacity=0.6
    )
)
base.add_child(neighbourhoods)

In [ ]:
boston_Marker=folium.Map(location=[lat,lang],zoom_start=12)
for lat,lng,num in zip(df.Y,df.X,range(1,df.shape[0])): 
         popup = folium.Popup(df['SCH_NAME'][num], parse_html=True)
         folium.Marker(
                    [lat,lng],
                    popup=popup,
                    icon=folium.Icon(color=colors[df.iloc[num,-8]])
            ).add_to(boston_Marker)
boston_Marker

In [ ]:
boston_cluster=folium.Map(location=[lat,lang],zoom_start=12)
from folium import plugins
cluster=plugins.MarkerCluster().add_to(boston_cluster)

boston_Marker=folium.Map(location=[lat,lang],zoom_start=12)
for lat,lng,num in zip(df.Y,df.X,range(1,df.shape[0])): 
         popup = folium.Popup(df['SCH_NAME'][num], parse_html=True)
         folium.Marker(
                    [lat,lng],
                    popup=popup,
                    icon=folium.Icon(color=colors[df.iloc[num,-8]])
            ).add_to(cluster)
boston_cluster
